# Google ADK + Gaugid Integration

This notebook demonstrates how to integrate Gaugid SDK with Google Agent Development Kit (ADK) to build production-ready multi-agent systems with personalized user context.

In [ ]:
import os
import asyncio
import google.generativeai as genai
from gaugid import GaugidClient

# Set your API keys
os.environ["GOOGLE_API_KEY"] = "your-key-here"
os.environ["GAUGID_CONNECTION_TOKEN"] = "your-token-here"

In [ ]:
async def load_user_context(client: GaugidClient) -> str:
    """Load user context from Gaugid profile using connection token mode."""
    # Connection token mode: No DID needed - profile resolved from token
    profile = await client.get_profile(
        scopes=["a2p:preferences", "a2p:professional", "a2p:context", "a2p:interests"]
    )
    
    # Extract memories from different memory types
    memories = profile.get("memories", {})
    episodic = memories.get("episodic", [])
    semantic = memories.get("semantic", [])
    procedural = memories.get("procedural", [])
    
    context_parts = []
    
    # Add episodic memories
    for memory in episodic:
        category = memory.get("category", "general")
        content = memory.get("content", "")
        context_parts.append(f"- [Episodic: {category}] {content}")
    
    # Add semantic memories
    for memory in semantic:
        category = memory.get("category", "general")
        content = memory.get("content", "")
        context_parts.append(f"- [Semantic: {category}] {content}")
    
    # Add procedural memories
    for memory in procedural:
        category = memory.get("category", "general")
        content = memory.get("content", "")
        context_parts.append(f"- [Procedural: {category}] {content}")
    
    return "\n".join(context_parts) if context_parts else "No user context available."

# Initialize clients
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
gaugid_client = GaugidClient(connection_token=os.environ["GAUGID_CONNECTION_TOKEN"])

# Load user context (connection token mode - no DID needed)
user_context = await load_user_context(gaugid_client)
print("User context loaded:")
print(user_context)

In [ ]:
# Create personalized agents with user context
base_system_prompt = f"""You are part of a multi-agent system that helps users with various tasks.

## USER PROFILE (from Gaugid)
{user_context}

## Guidelines
- Adapt your communication style to match user preferences
- Reference their expertise level when explaining concepts
- Consider their current projects and learning goals
- Be aware of their tool preferences
- Personalize examples and recommendations

Be helpful, technical, and precise. Adapt to the user's expertise level."""

# Research Agent
research_model = genai.GenerativeModel(
    model_name="gemini-2.5-flash",
    system_instruction=f"""{base_system_prompt}

You are a Research Specialist. Your role is to conduct thorough research on user topics.
You understand the user's interests, expertise level, and preferences.""",
)

# Writing Agent
writing_model = genai.GenerativeModel(
    model_name="gemini-2.5-flash",
    system_instruction=f"""{base_system_prompt}

You are a Content Writer. Your role is to create personalized content based on research.
You adapt your writing style to match the user's preferences and expertise level.""",
)

# Analysis Agent
analysis_model = genai.GenerativeModel(
    model_name="gemini-2.5-flash",
    system_instruction=f"""{base_system_prompt}

You are a Data Analyst. Your role is to analyze information and provide personalized insights.
You consider the user's professional background and interests when providing analysis.""",
)

print("✅ Multi-agent system ready with 3 specialized agents")

## Using GaugidMemoryService with ADK

For production use, you can use `GaugidMemoryService` as a `MemoryService` implementation
that integrates with Google ADK's memory system. This allows agents to:
- Store session information in Gaugid profiles
- Search Gaugid memories for relevant context
- Use PreloadMemoryTool to automatically load memories

See `examples/frameworks/gaugid_memory_service.py` for the full implementation.

In [ ]:
# Use the research agent
user_input = "What are the latest trends in AI agent development?"

print(f"User: {user_input}")
print(f"\nResearch Agent processing...")

response = research_model.generate_content(user_input)
print(f"\nResearch Agent: {response.text}")

# Optionally, save the interaction as a memory
memory_result = await gaugid_client.propose_memory(
    content=f"User asked about AI agent development trends",
    category="a2p:professional",
    memory_type="episodic",
    confidence=0.9,
    context="Interaction with Google ADK research agent"
)
print(f"\nMemory saved: {memory_result.get('proposal_id')}")